In [ ]:
# Načítanie knižníc
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import decomposition
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import csv
from sklearn.metrics import silhouette_score

# Preprocesing a načítanie dát
def read_csv_file(file_path):
    data = []
    firstRow = True
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if firstRow is True:
                firstRow = False
            else:
                numerical_row = [float(value) for value in row[2:]]
                data.append(numerical_row)
    return data

file_path = "/content/sample_data/Data_prepared_final.csv"
data = read_csv_file(file_path)

# štandartizácia
Xscaler = MinMaxScaler()
Xpp = Xscaler.fit_transform(data)

# Inicializácia PCA s počtom komponentov rovným počtu parametrov
pca = PCA()

# Aplikácia PCA na dáta
data_pca = pca.fit(Xpp)

# Vysvetlená variancia
explained_variance = pca.explained_variance_ratio_

# Určíenie počtu zdrojov na základe prahu vysvetlenej variancie
threshold = 0.95  # Prah: 95% vysvetlenej variancie
cumulative_variance = np.cumsum(explained_variance)
num_sources = np.argmax(cumulative_variance >= threshold) + 1

print("Počet zdrojov:", num_sources)

# Vykreslenie grafu kumulatívnej vysvetlenej variance
plt.plot(np.arange(1, 10), cumulative_variance, marker='o', linestyle='-')
plt.xlabel('Počet komponentov')
plt.ylabel('Kumulatívna vysvetlená variancia')
plt.title('Kumulatívna vysvetlená variancia')
plt.grid(True)
plt.show()

In [3]:
# Aplikácia PCA pre 4 komponenty
pca = PCA(n_components = 4)
pca.fit(Xpp)
data_pca_final = pca.transform(Xpp)

In [ ]:
# Silhouette metóda
max_clusters = 20   # preddefinovaný maximálny počet clustrov
silhouette_scores = []
for n_clusters in range(2, max_clusters+1):
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        cluster_labels = kmeans.fit_predict(data_pca_final)
        silhouette_avg = silhouette_score(data_pca_final, cluster_labels)
        silhouette_scores.append(silhouette_avg)
        print("Calculating clusters: " + str(n_clusters) + "/" + str(max_clusters + 1) + " - Silhouette score: " + str(silhouette_avg) + " - Best: " + str(np.argmax(silhouette_scores) + 2))

    # Find the optimal number of clusters with the highest silhouette score
optimal_clusters = np.argmax(silhouette_scores) + 2  # Add 2 because we started from 2 clusters

# Nájdenie optimálneho počtu zhlukov

print("Optimal number of clusters:", optimal_clusters)
print("Silhouette scores for different number of clusters:", silhouette_scores)

plt.figure(figsize=(8, 6))
plt.plot(range(2, max_clusters+1), silhouette_scores, marker='o')
plt.title('Silhouette metóda')
plt.xlabel('Počet zhlukov')
plt.ylabel('Silhouette skóre')
plt.grid(True)
plt.show()

In [ ]:
# Elbow metóda
inertia = []
for n_clusters in range(1, max_clusters+10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(data_pca_final)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, max_clusters+10), inertia, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()